In [158]:
import sys
import csv
!{sys.executable} -m pip install tensorflow
import tensorflow as tf
import numpy
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

### Given the data set, do a quick exploratory data analysis to get a feel for the distributions and biases of the data.  Report any visualizations and findings used and suggest any other impactful business use cases for that data.

In [83]:
file = open('XTern 2024 Artificial Intelegence Data Set - Xtern_TrainData.csv')

In [84]:
data = csv.DictReader(file)

In [85]:
years = [0, 0, 0, 0]
for row in data:
    match row['Year']:
        case "Year 1":
            years[0] += 1
        case "Year 2":
            years[1] += 1
        case "Year 3":
            years[2] += 1
        case "Year 4":
            years[3] += 1
years

[5, 2273, 2719, 3]

Mostly year 2 and year 3 students order from FoodX. This maybe due to cheap food options being able to Year 1 students at the University food hall or perhaps the FoodX foodtruck is only near the 2nd and 3rd year dormitories.

In [86]:
file.seek(0)
majors = []
majorsNum = []
for row in data:
    if not (row['Major'] in majors):
        majors.append(row['Major'])
        majorsNum.append(0)
    majorsNum[majors.index(row['Major'])] += 1
for i in range(len(majors) - 1):
    print(majors[i + 1], majorsNum[i + 1])

Physics 610
Chemistry 640
Biology 635
Business Administration 334
Anthropology 146
Mathematics 582
Economics 511
Astronomy 619
Marketing 239
Political Science 309
Finance 135
Sociology 31
Accounting 62
Psychology 76
International Business 29
Music 21
Mechanical Engineering 11
Philosophy 4
Fine Arts 3
Civil Engineering 3


The most common customers are Chemistry majors, thenBiology and then Physics. It seems that students of science are the most likely to order at FoodX.

In [87]:
file.seek(0)
unis = []
unisNum = []
for row in data:
    if not (row['University'] in unis):
        unis.append(row['University'])
        unisNum.append(0)
    unisNum[unis.index(row['University'])] += 1
for i in range(len(unis) - 1):
    print(unis[i + 1], unisNum[i + 1])

Indiana State University 1309
Ball State University 1085
Butler University 1614
Indiana University-Purdue University Indianapolis (IUPUI) 682
University of Notre Dame 144
University of Evansville 143
Valparaiso University 9
Purdue University 1
Indiana University Bloomington 12
DePauw University 1


FoodX is most popular with Butler University students, followed by IU students.

In [109]:
file.seek(0)
time = []
timeNum = []
for row in data:
    if not (row['Time'] in time):
        time.append(row['Time'])
        timeNum.append(0)
    timeNum[time.index(row['Time'])] += 1
time = time[1:]
timeNum = timeNum[1:]
for i in range(len(time)):
    time[i] = int(time[i])
timeZip = sorted(zip(time, timeNum))
timeZip

[(8, 8),
 (9, 40),
 (10, 247),
 (11, 857),
 (12, 1314),
 (13, 1316),
 (14, 883),
 (15, 282),
 (16, 49),
 (17, 4)]

11-14 is the most popular time of day to get food. 

In [92]:
file.seek(0)
orders = []
ordersNum = []
for row in data:
    if not (row['Order'] in orders):
        orders.append(row['Order'])
        ordersNum.append(0)
    ordersNum[orders.index(row['Order'])] += 1
for i in range(len(orders) - 1):
    print(orders[i + 1], ordersNum[i + 1])

Fried Catfish Basket 490
Sugar Cream Pie 512
Indiana Pork Chili 510
Indiana Corn on the Cob (brushed with garlic butter) 495
Indiana Buffalo Chicken Tacos (3 tacos) 496
Sweet Potato Fries 508
Ultimate Grilled Cheese Sandwich (with bacon and tomato) 503
Breaded Pork Tenderloin Sandwich 494
Cornbread Hush Puppies 510
Hoosier BBQ Pulled Pork Sandwich 482


The most popular item on the menu is Sugar Cream Pie. This might imply that students are looking for a quick sweet snack from FoodX.

In [240]:
file.close()

### Consider implications of data collection, storage, and data biases you would consider relevant here considering Data Ethics, Business Outcomes, and Technical Implications

Ethically as long as the data is held securely and the names and identities of the students using the app are not disclosed within the data there are few ethical concerns. The data does not discriminate based on identity and simply reports the student's academic major, year and order details. 

Through analyzing the data we can see a few Business implications. There is most likely no easily available FoodX truck in Purdue, IU Bloomington, DePauw, and Valparaiso. I make this prediction because there there are less than 12 orders from students of these universities. The overwhelming majority of orders occur between 11-14. If the costs outweigh the profits it could be prudent to reduce the amount of hours the food truck operates. Especially at 17 and onwards the amount of orders is tiny. The most popular item on the menu is a Sugar Cream Pie, this is the only desert item on the menu and it might improve sales to include more sugary items. 

There is little to no data on the preferences of 1st years and 2nd years. Most likely the NN will be much more innacurate for 1st year or 4th years students. A similar problem arises with students of the 4 universities outlined above which had few responses. I believe time of day and major are going to be the major indicators of preference, students most likely buy heavier meals during lunch time and lighter snacks at odd hours. The Neural network will have 10 input neurons to represent the University of the student, 10 input neurons for the Major, a single neuron between 0 and 1 that represents the time of day, and 4 neurons to represent year. This makes a total of 35 input neurons. There shall also be 10 ouput neurons for each food item. 

### Build a model to predict a customers order from their available information.  You will be graded largely on your intent and process when designing the model, performance is secondary. It is strongly suggested that you use SKLearn for this model as to not take too much time.  You may use any kind implementation you would like though, but it must be pickelable and have a “.predict()” method similar to SKLearn

Building the Neural Network I will be using tensorflow, as it is the library I am most familiar with. I created two simple functions that extract the x and y values from the CSV file. After analyzing the data I have determined that the best way to categorize the x and y is as follows. The x will be 35 input neurons. 4 Neurons to represent the Year of the customer, 10 to reperesent their University, 20 to represent their major and a single Neuron to represent the time they ordered from FoodX. I made this choice because while Year, University, and Major are discrete choices that the student has made the time of order lies on a spectrum and as such would be better represented through a single neuron.

The output is 10 neurons representing the 10 different orders that the student could make. 1 represents the Order that was actually ordered, all of the other orders will be 0. When fitting the program the loss function has been set to Logits=True, as such it will treat the output as probablities that the program believes one specific choice is the correct one over another. The highest number will be treated as the Network's choice and will be used to determine the accuracy. After this I have split the x and y set into training and testing data which consist of entirely sepparate cases. I will be testing the accuracy of the Neural network on the test data after the training is complete to account for overfitting. I have also included a Dropout layer within the neural network to reduce overfitting. 

In [241]:
def get_x(fileName):
    x = []
    file = open(fileName)
    data = csv.DictReader(file)
    next(data)
    for row in data:
        year = row['Year']
        match year:
            case "Year 1":
                year = [1, 0, 0, 0]
            case "Year 2":
                year = [0, 1, 0, 0]
            case "Year 3":
                year = [0, 0, 1, 0]
            case "Year 4":
                year = [0, 0, 0, 1]
        major = row['Major']
        match major:
            case "Physics":
                major = [0] * 20
                major[0] += 1
            case "Chemistry":
                major = [0] * 20
                major[1] += 1
            case "Biology":
                major = [0] * 20
                major[2] += 1
            case "Business Administration":
                major = [0] * 20
                major[3] += 1
            case "Anthropology":
                major = [0] * 20
                major[4] += 1
            case "Mathematics":
                major = [0] * 20
                major[5] += 1
            case "Economics":
                major = [0] * 20
                major[6] += 1
            case "Astronomy":
                major = [0] * 20
                major[7] += 1
            case "Marketing":
                major = [0] * 20
                major[8] += 1
            case "Political Science":
                major = [0] * 20
                major[9] += 1
            case "Finance":
                major = [0] * 20
                major[10] += 1
            case "Sociology":
                major = [0] * 20
                major[11] += 1
            case "Accounting":
                major = [0] * 20
                major[12] += 1
            case "Psychology":
                major = [0] * 20
                major[13] += 1
            case "International Business":
                major = [0] * 20
                major[14] += 1
            case "Music":
                major = [0] * 20
                major[15] += 1
            case "Mechanical Engineering":
                major = [0] * 20
                major[16] += 1
            case "Philosophy":
                major = [0] * 20
                major[17] += 1
            case "Fine Arts":
                major = [0] * 20
                major[18] += 1
            case "Civil Engineering":
                major = [0] * 20
                major[19] += 1
        university = row['University']
        match university:
            case "Indiana State University":
                university = [0] * 10
                university[0] += 1
            case "Ball State University":
                university = [0] * 10
                university[1] += 1
            case "Butler University":
                university = [0] * 10
                university[2] += 1
            case "Indiana University-Purdue University Indianapolis (IUPUI)":
                university = [0] * 10
                university[3] += 1
            case "University of Notre Dame":
                university = [0] * 10
                university[4] += 1
            case "University of Evansville":
                university = [0] * 10
                university[5] += 1
            case "Valparaiso University":
                university = [0] * 10
                university[6] += 1
            case "Purdue University":
                university = [0] * 10
                university[7] += 1
            case "Indiana University Bloomington":
                university = [0] * 10
                university[8] += 1
            case "DePauw University":
                university = [0] * 10
                university[9] += 1
        time = int(row['Time'])
        time /= 24
        time = [time]
        x.append(numpy.array(year + major + university + time))
    x = numpy.array(x)
    file.close()
    return x

In [242]:
def get_y(fileName):
    y = []
    file = open(fileName)
    data = csv.DictReader(file)
    next(data)
    for row in data:
        order = [0] * 10
        match row['Order']:
            case "Fried Catfish Basket":
                order[0] += 1
            case "Sugar Cream Pie":
                order[1] += 1
            case "Indiana Pork Chili":
                order[2] += 1
            case "Indiana Corn on the Cob (brushed with garlic butter)":
                order[3] += 1
            case "Indiana Buffalo Chicken Tacos (3 tacos)":
                order[4] += 1
            case "Sweet Potato Fries":
                order[5] += 1
            case "Ultimate Grilled Cheese Sandwich (with bacon and tomato)":
                order[6] += 1
            case "Breaded Pork Tenderloin Sandwich":
                order[7] += 1
            case "Cornbread Hush Puppies":
                order[8] += 1
            case "Hoosier BBQ Pulled Pork Sandwich":
                order[9] += 1
        y.append(numpy.array(order))
    y = numpy.array(y)
    file.close()
    return y

In [243]:
x = get_x('XTern 2024 Artificial Intelegence Data Set - Xtern_TrainData.csv')
y = get_y('XTern 2024 Artificial Intelegence Data Set - Xtern_TrainData.csv')

In [244]:
x_train = x[int(len(x) / 2):]
x_test = x[:int(len(x) / 2)]
y_train = y[int(len(y) / 2):]
y_test = y[:int(len(y) / 2)]

In [245]:
model = tf.keras.Sequential([
        tf.keras.layers.Dense(35),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.08),
        tf.keras.layers.Dense(10)
])

In [246]:
model.compile(optimizer='adam', 
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), 
              metrics=['accuracy'])

In [247]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
79/79 [==============================] - 1s 2ms/step - loss: 1.9759 - accuracy: 0.3556
Epoch 2/5
79/79 [==============================] - 0s 2ms/step - loss: 1.4961 - accuracy: 0.4848
Epoch 3/5
79/79 [==============================] - 0s 2ms/step - loss: 1.3996 - accuracy: 0.5008
Epoch 4/5
79/79 [==============================] - 0s 2ms/step - loss: 1.3576 - accuracy: 0.5100
Epoch 5/5
79/79 [==============================] - 0s 2ms/step - loss: 1.3109 - accuracy: 0.5232


In [248]:
loss, acc = model.evaluate(x_test, y_test, verbose=2)
print("Test Accuracy:", acc)

79/79 - 0s - loss: 1.3208 - accuracy: 0.5214 - 263ms/epoch - 3ms/step
Test Accuracy: 0.5214085578918457


### Given the work required to bring a solution like this to maturity and its performance, what considerations would you make to determine if this is a suitable course of action?

Currently the system is performing at 52% accuracy for guessing between 10 possible orders. This is quite good compared to random chance, however if the company is losing 10% of the revenue on every order it guesses incorrectly I do not believe the system with the current accuracy is acceptable. More data and a larger Neural Network would be required to guess orders with better than 75% accuracy. With the current metrics measured I do not believe it is reasonable to hope for a significantly more precise system. Perhaps if the NN was trained on specific user's demographics and previous orders it would be more accurate. Knowing the age, race, interests, and past orders and time ordered of the student would be very helpful in determining their future orders. Currently however it would most likely lose the company more money than acceptable. The idea either needs to be abbandoned or much more user data needs to be scraped from data brokers or their activity on the app. 